In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window


spark = SparkSession.builder.appName("weather_data").master("local[*]").getOrCreate()

dataframe = spark.read.csv(
    "/content/sample_data/weather_data.csv", header=True, inferSchema=True
)

# Преобразуйте столбец date в формат даты.
dataframe = dataframe.withColumn("date", F.to_date(F.col("date"), "yyyy-MM-dd"))

# Заполнение пропущенных значений средними значениями по метеостанциям
dataframe = dataframe.withColumn(
    "temperature",
    F.when(F.col("temperature").isNotNull(), F.col("temperature")).otherwise(
        F.avg("temperature").over(Window.partitionBy("station_id"))
    ),
)

dataframe = dataframe.withColumn(
    "precipitation",
    F.when(F.col("precipitation").isNotNull(), F.col("precipitation")).otherwise(
        F.avg("precipitation").over(Window.partitionBy("station_id"))
    ),
)

dataframe = dataframe.withColumn(
    "wind_speed",
    F.when(F.col("wind_speed").isNotNull(), F.col("wind_speed")).otherwise(
        F.avg("wind_speed").over(Window.partitionBy("station_id"))
    ),
)


dataframe.show()